In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/1016/Project/Typicality_project/

/content/drive/MyDrive/1016/Project/Typicality_project


In [3]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from scipy import stats
import ast 

In [4]:
categories = ['banana','beach','car','church','lighthouse','mountain','mug','plane']
path='./Analysis/human_rating_bycat'

In [5]:
for i, cate in enumerate(categories):
    col = [cate+'_'+str(i+1) for i in range(16)]
    new = pd.read_csv(path+'/'+cate+'.csv')
    new.columns = col
    if i == 0:
        old = new
    else:
        old = pd.concat([old,new],axis=1)
df = old

In [6]:
df.head()

,banana_1,banana_2,banana_3,banana_4,banana_5,banana_6,banana_7,banana_8,banana_9,banana_10,banana_11,banana_12,banana_13,banana_14,banana_15,banana_16,beach_1,beach_2,beach_3,beach_4,beach_5,beach_6,beach_7,beach_8,beach_9,beach_10,beach_11,beach_12,beach_13,beach_14,beach_15,beach_16,car_1,car_2,car_3,car_4,car_5,car_6,car_7,car_8,...,mountain_9,mountain_10,mountain_11,mountain_12,mountain_13,mountain_14,mountain_15,mountain_16,mug_1,mug_2,mug_3,mug_4,mug_5,mug_6,mug_7,mug_8,mug_9,mug_10,mug_11,mug_12,mug_13,mug_14,mug_15,mug_16,plane_1,plane_2,plane_3,plane_4,plane_5,plane_6,plane_7,plane_8,plane_9,plane_10,plane_11,plane_12,plane_13,plane_14,plane_15,plane_16
0,7,5,6,5,5,7,7,6,6,4,2,3,7,4,4,6,4,4,7,6,7,6,7,4,6,7,3,3,5,5,4,1,6,7,6,7,7,4,4,7,...,7,5,7,7,2,7,7,6,4,5,3,5,4,5,7,7,2,7,6,7,5,7,7,5,7,5,7,5,5,7,7,3,7,7,2,5,6,4,4,7
1,7,6,7,6,6,6,6,6,6,6,5,4,6,7,4,6,4,5,5,5,5,5,7,5,5,5,4,5,2,6,4,5,4,6,6,4,5,1,1,7,...,5,5,6,6,4,4,5,5,5,6,1,5,5,6,6,6,2,5,5,6,2,6,6,5,6,5,5,3,5,5,7,3,4,7,3,6,5,5,3,7
2,7,7,7,6,6,7,7,7,7,7,2,5,7,7,4,7,5,7,7,7,7,7,7,5,7,7,5,6,7,7,5,5,6,7,7,7,7,5,4,7,...,7,4,7,7,6,7,6,7,6,7,5,4,5,7,7,5,3,7,7,6,5,7,7,6,7,6,7,4,7,7,7,4,6,7,4,5,6,5,5,7
3,7,7,7,2,3,7,7,5,5,4,1,2,6,5,1,5,3,5,4,4,7,5,6,3,6,7,4,3,3,2,3,2,6,7,6,5,7,3,2,7,...,7,3,4,6,4,5,3,6,1,4,2,3,3,6,7,5,1,6,7,4,5,5,5,4,7,5,6,1,3,6,7,2,4,7,2,3,3,1,2,7
4,7,6,7,5,5,7,7,6,6,5,4,4,7,7,4,5,5,5,6,6,7,7,7,6,6,7,5,5,6,5,5,4,5,7,6,5,5,5,4,5,...,6,4,6,6,4,6,7,7,4,4,3,4,5,5,6,7,4,6,5,7,7,6,6,6,7,4,5,4,4,6,6,4,5,6,4,4,5,3,4,7


(Test) Use quantiles to remove outliers 

In [7]:
quantile_low = df.quantile(0.1)
quantile_high = df.quantile(0.9)
iqr = quantile_high - quantile_low

In [8]:
outliers = (df < quantile_low) | (df > quantile_high)
df_new = df[outliers.sum(axis=1) <= 12]

Compute 7 scorse and sort images by order in survey

In [9]:
# header_list = ["type", "dir", "id_labels", "readable_labels", "id_label", "prob", "readable_label"]
# overfeat_df=pd.read_csv('./Overfeat/overfeat_scores_final.csv',index_col=0,header=0,names=header_list)

In [10]:
header_list = ["type", "dir", "id_labels", "readable_labels", "id_label", "prob", "readable_label"]
overfeat_df=pd.read_csv('./Overfeat/overfeat_scores_final.csv',index_col=0,header=0,names=header_list)
overfeat_df['typicality_score'] = np.round(overfeat_df['prob'] * 100,2)
overfeat_df['7_score']= np.round(overfeat_df['typicality_score']*7/100,2)
overfeat_df['type'] = overfeat_df['type'].apply(lambda x: x.lower())

In [11]:
overfeat_df['number'] = overfeat_df['dir'].astype('int')
overfeat_df = overfeat_df.sort_values(['type','number']).reset_index(drop=True)

In [12]:
overfeat_df.head()

,type,dir,id_labels,readable_labels,id_label,prob,readable_label,typicality_score,7_score,number
0,banana,1,"{'07753592-n': 0.980348, '07717410-n': 0.00572...","{'banana': 0.980348, 'acorn squash': 0.0057260...",07753592-n,0.980348,banana,98.03,6.86,1
1,banana,2,"{'02606052-n': 0.41746099999999997, '07753592-...","{'rock beauty, Holocanthus tricolor': 0.417460...",07753592-n,0.123135,banana,12.31,0.86,2
2,banana,3,"{'07753592-n': 0.41874700000000004, '03065424-...","{'banana': 0.41874700000000004, 'coil, spiral,...",07753592-n,0.418747,banana,41.87,2.93,3
3,banana,4,"{'07753592-n': 0.618004, '07720875-n': 0.14292...","{'banana': 0.618004, 'bell pepper': 0.142925, ...",07753592-n,0.618004,banana,61.80,4.33,4
4,banana,5,"{'07753592-n': 0.364579, '07930864-n': 0.12127...","{'banana': 0.364579, 'cup': 0.121272, 'coil, s...",07753592-n,0.364579,banana,36.46,2.55,5


# Correlation with human ratings

Changes from original script: 
1. Use spearman rank correlation instead of pearson
2. Compute average human ratings first, and then compute correlation with neural net typicality scores

without dropping outliers

In [13]:
final_corr = dict()
for t in np.unique(overfeat_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    human_ratings = df[col_names] 
    net_ratings = overfeat_df[overfeat_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
#     net_scores = net_ratings['readable_labels'].apply(lambda x: ast.literal_eval(x)[list(ast.literal_eval(x).keys())[0]]).values
#     net_scores = net_ratings['7_score'].values # both get worse scores

    net_scores = net_ratings['typicality_score'].values
    
    coef, p=stats.spearmanr(human_scores, net_scores)
#     coef, p=stats.pearsonr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.7099648397193359
beach:  0.40470945467236635
car:  0.44542966659981903
church:  0.38410606425268223
lighthouse:  0.6147058823529412
mountain:  0.4930096993281554
mug:  0.3006633093458874
plane:  0.5682670754510278


In [14]:
print(np.mean(list(final_corr.values())))

0.49010699896527693


after dropping outliers

In [15]:
final_corr = dict()
for t in np.unique(overfeat_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    
    human_ratings = df_new[col_names] 
    net_ratings = overfeat_df[overfeat_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
    coef, p=stats.spearmanr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.6804444721634385
beach:  0.39572597333024884
car:  0.5037180695877399
church:  0.32818257597068257
lighthouse:  0.5949932815016497
mountain:  0.4892675634202017
mug:  0.3165680473372781
plane:  0.544378698224852


In [16]:
print(np.mean(list(final_corr.values())))

0.4816598351920114


In [17]:
overfeat_df['human_score'] = np.array(df.apply(np.mean,axis=0))

In [18]:
overfeat_df.to_csv('overfeat_result.csv')